In [9]:
import ollama

prompt = "Who is mark zuckerberg?"

respone = ollama.chat(model="llama3.2", messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
])

respone['message']['content']

'Mark Zuckerberg is an American technology entrepreneur and philanthropist. He is best known for co-founding and leading the social media platform Facebook, as well as its subsequent acquisition of Instagram and WhatsApp.\n\nBorn on May 14, 1984, in White Plains, New York, Zuckerberg developed an interest in computers at a young age. He attended Phillips Exeter Academy and later enrolled at Harvard University, where he studied computer science and psychology.\n\nIn 2004, while still a sophomore at Harvard, Zuckerberg launched "Facemash," a website that allowed users to compare the photos of two students and vote on which one was more attractive. The site became popular, but also generated controversy and was eventually shut down by the university.\n\nLater that year, Zuckerberg launched "Thefacebook" from his college dorm room, initially targeting Harvard students. The platform quickly gained popularity, and it expanded to other colleges and universities before opening up to the genera

In [5]:
import psycopg2

conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5433" # Adjust the port if necessary
)

cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT NOT NULL,
        embedding vector(1024)
    )
""")
conn.commit()
cur.close()
conn.close()

In [14]:
from sentence_transformers import SentenceTransformer

conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5433" # Adjust the port if necessary
)
cur = conn.cursor()

embedder = SentenceTransformer('BAAI/bge-m3', device='cpu')

def add_document(text):
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()

document = [
    "ประเทศไทยมีอากาศร้อนชื้นตลอดทั้งปี",
    "วันนี้ฉันไปเดินเล่นที่สวนจตุจักรและซื้อของฝาก",
    "เทคโนโลยีปัญญาประดิษฐ์กำลังเปลี่ยนแปลงโลกอย่างรวดเร็ว",
    "อาหารไทยที่ฉันชอบมากที่สุดคือต้มยำกุ้ง",
    "การเรียนรู้ตลอดชีวิตเป็นกุญแจสำคัญสู่ความสำเร็จ",
    "การออกกำลังกายอย่างสม่ำเสมอช่วยให้สุขภาพดีขึ้น",
    "หนังเรื่องโปรดของฉันคือ 'พี่มาก...พระโขนง'",
    "ประเทศไทยมีวัฒนธรรมที่หลากหลายและสวยงาม",
    "เครื่องบินลำนี้จะออกเดินทางเวลา 9 โมงเช้า",
    "ภาษาไทยมีความซับซ้อนและไพเราะในเวลาเดียวกัน"
]

for doc in document:
    add_document(doc)

cur = conn.cursor()
conn.close()

In [17]:
def query_postgres(query, top_k=5):
    query_embedding = embedder.encode(query).tolist()

    conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5433" # Adjust the port if necessary
    )
    cur = conn.cursor()

    query_embedding_str = "[" + ",".join(map(str, query_embedding)) + "]"
    
    sql_query = """
    SELECT content, embedding <=> %s::vector AS similarity_score from documents
    ORDER BY similarity_score ASC LIMIT %s;
    """

    cur.execute(sql_query, (query_embedding_str, top_k))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

results = query_postgres("ตอน 9 โมงเช้าจะมีอะไรเกิดขึ้น")
results


[('เครื่องบินลำนี้จะออกเดินทางเวลา 9 โมงเช้า', 0.28161695222652017),
 ('เทคโนโลยีปัญญาประดิษฐ์กำลังเปลี่ยนแปลงโลกอย่างรวดเร็ว', 0.5528228878974915),
 ('การออกกำลังกายอย่างสม่ำเสมอช่วยให้สุขภาพดีขึ้น', 0.5886340386489599),
 ('วันนี้ฉันไปเดินเล่นที่สวนจตุจักรและซื้อของฝาก', 0.6022966504096985),
 ('ภาษาไทยมีความซับซ้อนและไพเราะในเวลาเดียวกัน', 0.6682782869504855)]

In [18]:
import ollama

def generate_response(query_text):
    retrieved_docs = query_postgres(query_text)

    context = "\n".join([doc[0] for doc in retrieved_docs])
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}\nAnswer:"
    respone = ollama.chat(model="llama3.2", messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ])

    return respone['message']['content']

generate_response("ตอน 9 โมงเช้าจะมีอะไรเกิดขึ้น")

'เครื่องบินลำนี้จะออกเดินทาง'